In [72]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import SentenceTransformerEmbeddings

load_dotenv()

True

In [33]:
#Document ingestion
video_id = "nAmC7SoVLd8"

api = YouTubeTranscriptApi()

try:
    transcript = api.fetch(video_id, languages=['en'])
    transcripts = transcript.to_raw_data()
    transcript_text = " ".join([entry['text'] for entry in transcripts])
except TranscriptsDisabled:
    print("Transcripts are disabled for this video.")
    transcript_text = ""

print(transcript_text)

Langston is a framework that allows you to build applications on top of llm or large language model in this crash course video we are going to go over all the basics of Lang chain and then we will build a restaurant either generator application using streamlit where you can input any cuisine Indian Mexican Etc and it will generate a fancy restaurant name along with the manual items first let us understand what is Lang chain and what kind of problem does it addressed when you are using chat GPD as an application internally it is making call to open AI API which internally uses any llm such as GPD 3.5 or 4. in this case cat GP itself is not an llm it is an application whereas GPD 3.54 these are large language models now let's say you want to build an application for restaurant idea generator where you give a Cuisine and then it will generate a fancy name such as Taco Temptation for Mexican and menu items as well let's say you give Indian Cuisine it will say Okay Curry Curry Palace or Sah

In [34]:
text_spliter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_spliter.create_documents([transcript_text])

In [37]:
len(chunks)

46

In [40]:
chunks[0]

Document(metadata={}, page_content="Langston is a framework that allows you to build applications on top of llm or large language model in this crash course video we are going to go over all the basics of Lang chain and then we will build a restaurant either generator application using streamlit where you can input any cuisine Indian Mexican Etc and it will generate a fancy restaurant name along with the manual items first let us understand what is Lang chain and what kind of problem does it addressed when you are using chat GPD as an application internally it is making call to open AI API which internally uses any llm such as GPD 3.5 or 4. in this case cat GP itself is not an llm it is an application whereas GPD 3.54 these are large language models now let's say you want to build an application for restaurant idea generator where you give a Cuisine and then it will generate a fancy name such as Taco Temptation for Mexican and menu items as well let's say you give Indian Cuisine it wil

In [39]:
chunks[0].page_content

"Langston is a framework that allows you to build applications on top of llm or large language model in this crash course video we are going to go over all the basics of Lang chain and then we will build a restaurant either generator application using streamlit where you can input any cuisine Indian Mexican Etc and it will generate a fancy restaurant name along with the manual items first let us understand what is Lang chain and what kind of problem does it addressed when you are using chat GPD as an application internally it is making call to open AI API which internally uses any llm such as GPD 3.5 or 4. in this case cat GP itself is not an llm it is an application whereas GPD 3.54 these are large language models now let's say you want to build an application for restaurant idea generator where you give a Cuisine and then it will generate a fancy name such as Taco Temptation for Mexican and menu items as well let's say you give Indian Cuisine it will say Okay Curry Curry Palace or"

In [45]:

embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
vector_store = FAISS.from_documents(chunks, embeddings)

C:\Users\yadav\AppData\Local\Temp\ipykernel_16748\1633633582.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')


In [47]:
vector_store.index_to_docstore_id

{0: 'f6088636-a54c-4b76-8ec8-4ae991ec24dd',
 1: '184b6b3a-00df-405d-ae14-f3ca718c613e',
 2: '0430976d-86dd-4688-a988-51a3c0766e02',
 3: 'd6698721-6476-477b-b190-92d35b335708',
 4: 'b93cb443-54ca-423e-860e-3032d227034a',
 5: 'aaac1747-f769-4643-a57b-df581de95e44',
 6: '80c0c1be-915a-499a-9c0e-ba0f1071fedb',
 7: 'ca7bc52e-7f30-4e60-a281-ab99e1b75f1e',
 8: 'bbb8e8c8-3ea3-4498-9281-1ed54e6870ba',
 9: '011b7b3e-572e-4885-b8b8-0989605d279c',
 10: 'fc1d7e8f-e8d4-41b5-8c77-d9dbbbe0e5f0',
 11: '02ffaf59-c1b7-4d06-9bce-090096f9a8ac',
 12: '4dacd084-6523-44d3-abdb-edea95c6dd1d',
 13: 'b20141f3-5f92-44c4-bdc2-292744cc1667',
 14: '4ab0c1fd-6bee-42c5-8638-b678ab506f62',
 15: '76506849-39b9-413d-91d2-990e8c7193bc',
 16: 'c918b121-dbef-43c1-9833-d50296ad1933',
 17: '7bc3372d-bab4-4c1a-9c4f-16e081427c54',
 18: 'a502f0da-95c6-49ab-9545-80485e568516',
 19: '0ffbe061-5b1f-49b0-81f4-3eaea958a664',
 20: '50caffd0-63d9-4fe5-bfaf-f06590a5551a',
 21: 'a2efdda0-0de3-4fd1-90f1-0db206a91dbd',
 22: '05dd2585-ca67-

In [48]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [49]:
retriever.invoke("what is langchain?")

[Document(id='f6088636-a54c-4b76-8ec8-4ae991ec24dd', metadata={}, page_content="Langston is a framework that allows you to build applications on top of llm or large language model in this crash course video we are going to go over all the basics of Lang chain and then we will build a restaurant either generator application using streamlit where you can input any cuisine Indian Mexican Etc and it will generate a fancy restaurant name along with the manual items first let us understand what is Lang chain and what kind of problem does it addressed when you are using chat GPD as an application internally it is making call to open AI API which internally uses any llm such as GPD 3.5 or 4. in this case cat GP itself is not an llm it is an application whereas GPD 3.54 these are large language models now let's say you want to build an application for restaurant idea generator where you give a Cuisine and then it will generate a fancy name such as Taco Temptation for Mexican and menu items as w

In [ ]:
prompt = PromptTemplate(
    template = "Use the following pieces of context to answer the question at the end. If you don't know the answer, say you don't know. {context} Question: {question}",
    input_variables=["context", "question"]
)

In [57]:
question = "what is the video about? and what is langchain?"
retrieved_docs = retriever.invoke(question)
retrieved_docs

[Document(id='6261b627-d2d4-46af-adde-bc315e93ad8e', metadata={}, page_content='or sharing this video might be a smaller food for you but it would mean a big thing for us it will help us get this video to more people so that those people can also benefit and we also get some appreciation some reward of the hard work that we are putting and if you have any question Post in the comment box below the code link is given in the video description thank you for watching bye bye thank you'),
 Document(id='85e48db2-9529-4b3b-a799-27b01a4282e8', metadata={}, page_content="which game you are talking about which particular match you are talking about okay so here I know it's probably not the best example but the idea is I wanted to demonstrate this K parameter here and based on this use case uh you might see benefit in using conversational buffer window memory that's all we had for this video in the future we are going to build end-to-end llm applications using some of the advanced features such a

In [60]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)

In [61]:
context_text

"or sharing this video might be a smaller food for you but it would mean a big thing for us it will help us get this video to more people so that those people can also benefit and we also get some appreciation some reward of the hard work that we are putting and if you have any question Post in the comment box below the code link is given in the video description thank you for watching bye bye thank you\n\nwhich game you are talking about which particular match you are talking about okay so here I know it's probably not the best example but the idea is I wanted to demonstrate this K parameter here and based on this use case uh you might see benefit in using conversational buffer window memory that's all we had for this video in the future we are going to build end-to-end llm applications using some of the advanced features such as retrieval QA chain the face or DB store Vector store uh things like that so if you want to get notified about upcoming llml line chain videos what you can do

In [65]:
final_prompt = prompt.invoke(input={"context": context_text, "question": question})

In [67]:
print(final_prompt)

text="Use the following pieces of context to answer the question at the end. If you don't know the answer, say you don't know. or sharing this video might be a smaller food for you but it would mean a big thing for us it will help us get this video to more people so that those people can also benefit and we also get some appreciation some reward of the hard work that we are putting and if you have any question Post in the comment box below the code link is given in the video description thank you for watching bye bye thank you\n\nwhich game you are talking about which particular match you are talking about okay so here I know it's probably not the best example but the idea is I wanted to demonstrate this K parameter here and based on this use case uh you might see benefit in using conversational buffer window memory that's all we had for this video in the future we are going to build end-to-end llm applications using some of the advanced features such as retrieval QA chain the face or 

In [68]:
model = ChatGroq(model="openai/gpt-oss-120b")

In [70]:
result=model.invoke(final_prompt)
print(result.content)

**What the video is about**

The video is a tutorial that walks through building a simple **Streamlit** application powered by a large language model (LLM) using the **LangChain** library.  
It covers several core concepts and features, such as:

* The **K‑parameter** and how it affects the model’s behavior.  
* **Conversational buffer window memory** – a way to keep recent chat history for the LLM.  
* An introduction to **agents**, which let the model decide which tools or actions to take.  
* A preview of more advanced LangChain components that will be covered in future videos, including **retrieval‑augmented generation (RAG)**, **vector stores**, and **FAISS/DB‑store** integration.  

The creator also asks viewers to like, share, and subscribe so the tutorial can reach a wider audience.

---

**What LangChain is**

**LangChain** is an open‑source Python framework for building applications that are driven by large language models.  
It provides high‑level building blocks that make i

#chains

In [75]:
def format_docs(retrieved_docs):
    context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return context_text

In [78]:
parallel_chain = RunnableParallel({
    "context" : retriever | RunnableLambda(format_docs),
    "question" : RunnablePassthrough()
})

In [79]:
parallel_chain.invoke(question)

{'context': "or sharing this video might be a smaller food for you but it would mean a big thing for us it will help us get this video to more people so that those people can also benefit and we also get some appreciation some reward of the hard work that we are putting and if you have any question Post in the comment box below the code link is given in the video description thank you for watching bye bye thank you\n\nwhich game you are talking about which particular match you are talking about okay so here I know it's probably not the best example but the idea is I wanted to demonstrate this K parameter here and based on this use case uh you might see benefit in using conversational buffer window memory that's all we had for this video in the future we are going to build end-to-end llm applications using some of the advanced features such as retrieval QA chain the face or DB store Vector store uh things like that so if you want to get notified about upcoming llml line chain videos wha

In [80]:
parser = StrOutputParser()

In [81]:
main_chain = parallel_chain | prompt | model | parser

In [85]:
result = main_chain.invoke("can you summarize the video?")

In [86]:
print(result)

**Video Summary**

The video walks through how to give a language‑model‑based chain “memory” so it can keep track of a conversation’s context. The presenter:

1. **Introduces the “K‑parameter”** – a setting used in the demo to illustrate how the size of the memory window affects the model’s responses.  
2. **Shows the default behavior of LangChain** – chains are stateless; the `memory` attribute is `None`, so the model forgets everything after each turn.  
3. **Demonstrates adding a conversational buffer memory** – by creating a `ConversationBufferMemory` (or similar) object and attaching it to the chain, the model can recall prior turns (e.g., remembering that the discussion is about cricket and answering follow‑up questions about the winning captain).  
4. **Explains why memory is useful** – especially for chat‑bot style applications where the user expects the system to retain the topic across multiple exchanges.  
5. **Hints at future videos** – upcoming tutorials will cover more ad